In [ ]:
from sbibm.algorithms import snle
from sbi_ebm.sbibm.tasks import get_task
from sbi_ebm.sbibm.sbibm_eval import snle
ret_snle_nsf = snle(task="MultiModalLikelihoodTask", num_samples=0, num_simulations=1000, num_observation=1, num_rounds=1, neural_net="nsf")
ret_snle_maf = snle(task="MultiModalLikelihoodTask", num_samples=0, num_simulations=1000, num_observation=1, num_rounds=1, neural_net="maf")

In [2]:
from sbi_ebm.sbibm.sbibm_smnle import smnle as run

ret = run(
    task="MultiModalLikelihoodTask",
    num_simulations=100000,
    num_rounds=1,
    num_observation=1,
    # observation=torch.zeros(1, 2),
    num_samples=1000,
    aux_MCMC_inner_steps_exchange_MCMC=10,
    aux_MCMC_proposal_size_exchange_MCMC=0.1,
    burnin_exchange_MCMC=10,
    bridging_exch_MCMC=0,
    propose_new_theta_exchange_MCMC="sphere",
    # technique="SSM",
    noise_sliced="sphere",
    technique="SSM",
    epochs=500,
    epochs_before_early_stopping=1000,
    batch_size=100000,
    automatic_transforms_enabled=False,
    lr_data=0.005,
    SM_lr_theta=0.01,
    cuda=True,
    no_bn=False,
    momentum=0.1,
    no_var_red_sliced=True,
    no_scheduler=True,
    scale_samples=True,
    scale_parameters=True,
    # theta_vect=theta_vect,
    use_jax_mcmc=True,
    seed=5,
)

data_transforms IndependentTransform(ComposeTransform(
    
), 1)
param_transforms IndependentTransform(ComposeTransform(
    
), 1)
Data generation took 0.2019 seconds


  0%|          | 0/11000 [00:00<?, ?it/s]

finding good initial position (doubly intractable)
finding good initial position
finding good initial position
good initial position found at:  Traced<ShapedArray(float32[2])>with<BatchTrace(level=1/1)> with
  val = Traced<ShapedArray(float32[100,2])>with<DynamicJaxprTrace(level=0/1)>
  batch_dim = 0
finding good initial position
finding good initial position
finding good initial position


In [21]:
from sbi_ebm.sbibm.sbi_ebm import run as run_unle
ret_aunle = run_unle(
    # "slcp", (10000,), 1,
    # "LDCT", (10000,10000), 1,
    # "slcp", (2000,2000,2000,2000,2000), 1,
    "MultiModalLikelihoodTask", (1000,), 1,
    num_smc_steps=20,
    num_mala_steps=3,
    use_warm_start=False,
    learning_rate=0.01,
    max_iter=200,
    weight_decay=0.1,
    random_seed=40,
    sampler="smc",
    num_particles=1000,
    batch_size=1000,
    restart_every=None,
    num_posterior_samples=1000,
    use_nuts=False,
    init_proposal="prior",
    noise_injection_val=0.0005,
    proposal="prior+noise",
    inference_sampler="smc",
    ebm_model_type="joint_tilted",
    select_based_on_test_loss=False,
    inference_proposal="prior",
    use_data_from_past_rounds=False,
)

using a network of width 50 and depth 4
generating data took time:  0:00:00
creating dataset with 1000 samples
z_scoring theta-part of sampling init dist
z_scoring theta-part of inference init dist
config.inference.sampling_init_dist=<numpyro.distributions.distribution.TransformedDistribution object at 0x7f1d08429640>
training likelihood model...
number of datasets:  1
first step...
....done.
iteration 10/200    : 7.705     unnormalized_train_log_l=-3.006     unnormalized_test_log_l=-2.918     train_log_l=-6.792    test_log_l=-6.704    ebm_log_l=-7.442    log_Z=0
iteration 20/200    : 0.803     unnormalized_train_log_l=-3.725     unnormalized_test_log_l=-3.657     train_log_l=-5.689    test_log_l=-5.621    ebm_log_l=-5.744    log_Z=0
iteration 30/200    : 0.893     unnormalized_train_log_l=-3.443     unnormalized_test_log_l=-3.420     train_log_l=-5.019    test_log_l=-4.996    ebm_log_l=-5.216    log_Z=0
iteration 40/200    : 1.471     unnormalized_train_log_l=-3.640     unnormalized_t

In [8]:
# import cloudpickle
# with open('ret_multimodal_snle_maf.pkl', 'wb') as f:
#     cloudpickle.dump(ret_snle_maf, f)
#     
# with open('ret_multimodal_snle_nsf.pkl', 'wb') as f:
#     cloudpickle.dump(ret_snle_nsf, f)
#     
# with open('ret_multimodal_smnle.pkl', 'wb') as f:
#     cloudpickle.dump(ret, f)
#     
# with open('ret_multimodal_aunle.pkl', 'wb') as f:
#     cloudpickle.dump(ret_aunle, f)
#     
# with open('ret_multimodal_sunle.pkl', 'wb') as f:
#     cloudpickle.dump(ret_sunle, f)

In [10]:
from density_utils.plotting import normalize_posterior, normalize_density
from sbi_ebm.distributions import maybe_wrap, maybe_wrap_log_l
from sbi_ebm.sbibm.tasks import JaxTask, get_task
import jax.numpy as jnp
smnle_normalized_posterior = normalize_posterior(
    # prior=maybe_wrap(ret.train_results.config.task.prior.log_prob),
    prior=JaxTask(get_task("MultiModalLikelihoodTask")).get_prior_dist().log_prob,
    likelihood=ret.train_results.single_round_results[0].jax_log_likelihood,
    x_obs=ret.train_results.config.task.x_obs[0],
    # bounds=((-10, 10), (-10, 10)),
    bounds=((-8, 8), (-8, 8)),
    nbins=101, log_space=True)

In [ ]:
import jax.numpy as jnp
import torch
from density_utils.plotting import plot_densities
from density_utils import plotting
import matplotlib.pyplot as plt
import torch

plt.style.use("paper.mplstyle")
plt.rcParams.update({
    "text.usetex": False,
    # "font.family": "Helvetica"
})

from sbi_ebm.sbibm.tasks import get_task, JaxTask
t = get_task("MultiModalLikelihoodTask")

f, axs  = plt.subplots(figsize=(20, 10), ncols=4, nrows=2)


_ = axs[0, 0].axis('off')

plot_densities({"sm": lambda x:  ret.train_results.single_round_results[0].jax_log_likelihood(jnp.zeros((2,)), x)}, bounds=((-5,5), (-5,5)), nbins=100, log_space=True, axs=axs[:, 0], batch_size=10)
plot_densities({"sm":  smnle_normalized_posterior}, bounds=((-4, 4), (-4,4)), nbins=100, log_space=True, axs=axs[1:, 0], batch_size=10)


_ = plotting.plot_density_pytorch(
((-4 ,4), (-4, 4)),
lambda x: (ret_snle_maf.train_results.posterior.flow.log_prob(theta=torch.zeros((2,)), x=x)[0]).exp(), nbins=100, ax=axs[0, 1]
)

_ = plotting.plot_density_pytorch(((-4 ,4), (-4, 4)), lambda theta: ret_snle_maf.train_results.posterior.flow.log_prob(theta)[0].exp(), nbins=100, ax=axs[1, 1])

plot_densities({"sm": lambda x:  ret_aunle.train_results.single_round_results[0].get_posterior(99)._log_likelihood(jnp.zeros((2,)), x)}, bounds=((-4, 4), (-4,4)), nbins=100, log_space=True, axs=axs[:, 2], batch_size=10)
plot_densities({"sm":  ret_aunle.train_results.single_round_results[0].get_posterior(99).log_prob}, bounds=((-4, 4), (-4,4)), nbins=100, log_space=True, axs=axs[1:, 2], batch_size=10)

plot_densities(
    {"sm":  lambda x: t._jax_log_likelihood(jnp.zeros((2,)), x)}, bounds=((-4, 4), (-4,4)), nbins=100, log_space=True, axs=axs[:, 3], batch_size=10
)

plot_densities(
    {"sm":  lambda theta: JaxTask(t).get_prior_dist().log_prob(theta) +  t._jax_log_likelihood(theta, jnp.zeros((2,)))}, bounds=((-4, 4), (-4,4)), nbins=100, log_space=True, axs=axs[1:, 3], batch_size=10
)
# plot_densities({"sm": lambda x:  ret.single_round_results[0].jax_log_likelihood(jnp.zeros((2,)), x)}, bounds=((-4,4), (-4,4)), nbins=40, log_space=True, axs=axs)

for rax in axs:
    for ax in rax:
        ax.axis('off')

axs[0, 0].text(x=0.5, y=1.2, size=24, horizontalalignment='center', s="SMNLE", transform=axs[0, 0].transAxes)
axs[0, 1].text(x=0.5, y=1.2, size=24, horizontalalignment='center', s="NLE", transform=axs[0, 1].transAxes)
axs[0, 2].text(x=0.5, y=1.2, size=24, horizontalalignment='center', s="AUNLE (Ours)", transform=axs[0, 2].transAxes, fontweight="bold")
axs[0, 3].text(x=0.5, y=1.2, size=24, horizontalalignment='center', s="Ground Truth", transform=axs[0, 3].transAxes)

axs[0, 0].text(x=-0.1, y=0.5, size=24, horizontalalignment='right', s="Likelihood", transform=axs[0, 0].transAxes)
axs[1, 0].text(x=-0.1, y=0.5, size=24, horizontalalignment='right', s="Posterior", transform=axs[1, 0].transAxes)

# f.savefig('multimodal_posterior_fig.png', dpi=300)